In [1]:
import pdfplumber
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# functions
def get_date(str_date):
    date = datetime.strptime(str_date+year, '%d-%b%Y')
    print("hello", date)
    return date

def is_date(date):
    try:
        get_date(date)
        return True
    except ValueError:
        return False

def is_float(str_float):
    try:
        float(str_float)
        if str_float[-3] == ".":
            return True
        else:
            return False
    except ValueError:
        return False

def get_amount():
    text[text.find(first_date):].split("\n")[0].split()[-1].replace(",","")

def enter_row(statement_dict,date,details,amount,balance):
    statement_dict["Date"].append(date)
    statement_dict["Transaction Details"].append(details)
    statement_dict["Amount"].append(amount)
    statement_dict["Balance"].append(balance)
    return statement_dict

def create_statement_dict():
    #create dictionary to hold temp values
    statement_dict= {
        "Date": [], 
        "Transaction Details": [], 
        "Amount": [],
        "Balance": [] 
    }
    statement_dict["Date"] = []
    statement_dict["Transaction Details"] = []
    statement_dict["Amount"] = []
    statement_dict["Balance"] = []
    return statement_dict

def transform_statement_to_df(statement):
    statement_dict = create_statement_dict()
    
    date = ""
    balance = ""
    amount = ""
    details = ""

    for lines in statement:
        items= lines.split()
        if is_float(items[-1].replace(",","")): #check if is not a new row of transaction details
            if details != "":
                if is_date(items[0]): #check if has date 
                    statement_dict = enter_row(statement_dict,date,details,amount,balance) #if so enter last row with balance
                    date = items[0] #reset date
                    if is_float(items[-2].replace(",","")): #check how many numeric there are
                        balance = items[-1].replace(",","") # if two then separate balance and amount
                        amount = items[-2].replace(",","")
                        details = " ".join(items[1:-2])
                    else:
                        amount = items[-1].replace(",","") # if only one then is amount
                        details = " ".join(items[1:-1])
                else: # if doesnt start with date
                    statement_dict = enter_row(statement_dict,date,details,amount,"") #enter prev row without updating balance
                    if is_float(items[-2].replace(",","")): #check how many numeric there are
                        balance = items[-1].replace(",","") # if two then separate balance and amount
                        amount = items[-2].replace(",","")
                        details = " ".join(items[:-2])
                    else:
                        amount = items[-1].replace(",","") # if only one then is amount
                        details = " ".join(items[:-1])
            else: 
                date = items[0]
                details = " ".join(items[1:-1])
                balance = items[-1].replace(",","")
        else:
            details = details + " " + " ".join(items)
    statement_dict = enter_row(statement_dict,date,details,"",balance)

    #put into df
    df = pd.DataFrame(statement_dict, columns= ["Date", "Transaction Details", "Amount", "Balance"])
    return df

def export_as_csv(df):
    df.to_csv("Statement "+str(get_date(last_date)).split()[0]+".csv", index=False)


In [3]:
# 1) Enter Invoice path, name formatting should be provided in 'statements/yyyy-mm.pdf' to work
invoice = 'statements/2016-06.pdf'
#read invoice pdf 
with pdfplumber.open(invoice) as pdf:
    page = pdf.pages[0]
    text = page.extract_text(x_tolerance=1)

In [4]:
# 2) run to check if statement is read with no errors and shows correct general details
year = invoice[invoice.find('/')+1:invoice.find('-')]

start = [i for i, s in enumerate(text.split("\n")) if 'B/F BALANCE' in s]
end = [i for i, s in enumerate(text.split("\n")) if 'C/F BALANCE' in s]
statement= text.split("\n")[start[0]:end[0]+1]

first_date = statement[0].split()[0]
last_date = statement[-1].split()[0]

if not(is_date(first_date) & is_date(last_date)):
    print("Statement Dates not found")
print("Statement date found = ", last_date, year, "\nPrev statement date = ",first_date, year)

hello 2016-04-12 00:00:00
hello 2016-06-11 00:00:00
Statement date found =  11-Jun 2016 
Prev statement date =  12-Apr 2016


In [5]:
# 3) run to get statement in dataframe format
dataframe = transform_statement_to_df(statement)
print("Statement", last_date, year)
dataframe

# 4) Optional export as csv
# export_as_csv(dataframe)

hello 2016-05-20 00:00:00
hello 2016-06-06 00:00:00
hello 2016-06-11 00:00:00
Statement 11-Jun 2016


,Date,Transaction Details,Amount,Balance
0,12-Apr,2016 B/F BALANCE,,5887.70
1,20-May,CASH DEPOSIT (20MAY16),400.00,
2,20-May,ATM WITHDRAWAL (20MAY16),500.00,
3,20-May,ATM WITHDRAWAL (20MAY16),3500.00,2287.70
4,6-Jun,CASH DEPOSIT (05JUN16),2500.00,4787.70
5,11-Jun,C/F BALANCE,,4787.70


Rules followed when transforming statement:
- steps check if ends in number 
- if no number then simply update details
- if end in number
- check if starts with date
- if not starts with date upload only the description and date then update only description
- if starts with date update balance with amount and upload with balance update the date and balance with the amt above
- diffrentiate for is amount if next line doesnt start with date, or if next line starts with date the second last is amount
